In [1]:
import torch
from alexnet.base import AlexNetBase
from tqdm import tqdm

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AlexNetBase().to(device)
model.load_state_dict(torch.load(".checkpoint/alexnet_5_epochs.pth"))

<All keys matched successfully>

In [ ]:
from data.dataset import get_dataloaders

_, test_dataloader, _ = get_dataloaders(batch_size=64, num_workers=4)

correct = 0
correct_top_5 = 0
total = 0

with torch.no_grad():
    with tqdm(total=len(test_dataloader.dataset) // test_dataloader.batch_size, desc=f"Testing Accuracy", unit=" batches") as pbar:
        model.eval()
        for batch in test_dataloader:
            images, labels = batch['pixel_values'], batch['labels']
            batch_size, num_crops = images.shape[:2]
            images = images.view(-1, *images.shape[2:]) # (batch_size * num_crops, channels, height, width)
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            outputs = outputs.view(batch_size, num_crops, -1)
            avg_outputs = outputs.mean(dim=1) # avg(each batch, :, each prediction)
            _, predicted = avg_outputs.max(1)

            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)
            _, top_5_predicted = avg_outputs.topk(5, dim=1)
            correct_top_5 += top_5_predicted.eq(labels.view(-1, 1)).sum().item()

            pbar.update(1)

print(f"Accuracy: {100 * correct / total:.2f}%")
print(f"Top-5 Accuracy: {100 * correct_top_5 / total:.2f}%")
print(f"Total samples: {total}")

Testing Accuracy:   0%|          | 0/781 [00:00<?, ? batches/s]

tensor(1, device='cuda:0')


Testing Accuracy:   0%|          | 0/781 [00:01<?, ? batches/s]

Accuracy: 39.06%
Top-5 Accuracy: 0.00%
Total samples: 64
